# 1st

In [ ]:
# Running with Bunnies

In [ ]:
# You and your rescued bunny prisoners need to get out of this collapsing death trap of a space
# station - and fast! Unfortunately, some of the bunnies have been weakened by their long imprisonment
# and can't run very fast. Their friends are trying to help them, but this escape would go a lot faster
# if you also pitched in. The defensive bulkhead doors have begun to close, and if you don't make it 
# through in time, you'll be trapped! You need to grab as many bunnies as you can and get through the 
# bulkheads before they close.
# The time it takes to move from your starting point to all of the bunnies and to the bulkhead will 
# be given to you in a square matrix of integers. Each row will tell you the time it takes to get to 
# the start, first bunny, second bunny, ..., last bunny, and the bulkhead in that order. The order of
# the rows follows the same pattern (start, each bunny, bulkhead). The bunnies can jump into your arms,
# so picking them up is instantaneous, and arriving at the bulkhead at the same time as it seals still
# allows for a successful, if dramatic, escape. (Don't worry, any bunnies you don't pick up will be able
#                                                to escape with you since they no longer have to carry 
#                                                the ones you did pick up.) You can revisit different
# spots if you wish, and moving to the bulkhead doesn't mean you have to immediately leave - you can 
# move to and from the bulkhead to pick up additional bunnies if time permits.
# In addition to spending time traveling between bunnies, some paths interact with the space station's 
# security checkpoints and add time back to the clock. Adding time to the clock will delay the closing
# of the bulkhead doors, and if the time goes back up to 0 or a positive number after the doors have 
# already closed, it triggers the bulkhead to reopen. Therefore, it might be possible to walk in a 
# circle and keep gaining time: that is, each time a path is traversed, the same amount of time is 
#     used or added.
# Write a function of the form answer(times, time_limit) to calculate the most bunnies you can pick 
# up and which bunnies they are, while still escaping through the bulkhead before the doors close 
# for good. If there are multiple sets of bunnies of the same size, return the set of bunnies with 
# the lowest prisoner IDs (as indexes) in sorted order. The bunnies are represented as a sorted list
# by prisoner ID, with the first bunny being 0. There are at most 5 bunnies, and time_limit is a 
# non-negative integer that is at most 999.
# For instance, in the case of
# [
# [0, 2, 2, 2, -1], # 0 = Start
# [9, 0, 2, 2, -1], # 1 = Bunny 0
# [9, 3, 0, 2, -1], # 2 = Bunny 1
# [9, 3, 2, 0, -1], # 3 = Bunny 2
# [9, 3, 2, 2, 0], # 4 = Bulkhead
# ]
# and a time limit of 1, the five inner array rows designate the starting point, bunny 0, bunny 1,
# bunny 2, and the bulkhead door exit respectively. You could take the path:
# Start End Delta Time Status
# - 0 - 1 Bulkhead initially open
# 0 4 -1 2
# 4 2 2 0
# 2 4 -1 1
# 4 3 2 -1 Bulkhead closes
# 3 4 -1 0 Bulkhead reopens; you and the bunnies exit
# With this solution, you would pick up bunnies 1 and 2. This is the best combination for this space
# station hallway, so the answer is [1, 2].
# Test cases
# ==========
# Inputs:
# (int) times = [[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]
# (int) time_limit = 3
# Output:
# (int list) [0, 1]
# Inputs:
# (int) times = [[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]]
# (int) time_limit = 1
# Output:
# (int list) [1, 2]

In [ ]:
#def solution(times, times_limit):
from copy import deepcopy
from itertools import permutations
# I am not happy with this implementation, due to a realization I have recently had:
#   My Bellman-Ford implementation is completely unnecessary. The Floyd algorithm can detect negative cycles, and
#   negative cycles are all I used Bellman-Ford for. Thus, with some refactoring, I could easily make this more
#   efficient.
def powerset(list):
    """
    :param list: The list to generate subsets of.
    :return: A generator that produces all subsets of this set.
    """
    x = len(list)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, list) if i & mask]
def getneighbourindex(neighbour, graphsize):
    if neighbour == "Bulkhead":
        return graphsize - 1
    elif neighbour == "Start":
        return 0
    else:
        return int(neighbour)+1
def matrix2graph(matrix):
    """
    This helper function turns our matrix into a graph that's a little easier to work with using Bellman-Ford.
    :param matrix: The original matrix.
    :return: matrix in dictionary format.
    """
    keys = ["Start"]
    for num in range(1, len(matrix)-1):
        keys.append(num-1)
    keys.append("Bulkhead")
    graph = dict(zip(keys, matrix))
    return graph
# Step 1: Initialize graph
def initialize(graph, source):
    """
    Step 1 of the Bellman-Ford algorithm.
    """
    distance = {}
    predecessor = {}
    for node in graph:
        # We start off assuming all nodes are too far away!
        distance[node] = 1000
        predecessor[node] = None
    distance[source] = 0 # For the source we know how to reach
    return distance, predecessor
def relax(node, neighbour, graph, distance, predecessor):
    nidx = getneighbourindex(neighbour, len(graph))
    if distance[node] + graph[node][nidx] < distance[neighbour]:
        distance[neighbour] = distance[node] + graph[node][nidx]
        predecessor[neighbour] = node
def bellman_ford(matrix, graph, times_limit, source):
    dist, pred = initialize(graph, source)
    for num in range(len(graph)-1):
        for node in graph:
            temp = dict(graph)
            del temp[node]
            for neighbour in temp:
                # Step 2: Relax edges repeatedly
                relax(node, neighbour, graph, dist, pred)
    # Step 3: Check for negative-weight cycles
    for node in graph:
        for neighbour in graph:
            nidx = getneighbourindex(neighbour, len(graph))
            if dist[node] + graph[node][nidx] < dist[neighbour]:
                # We found a negative cycle. Since the door is open forever, free all the bunnies!~
                return [num for num in range(0, len(graph)-2)]
    # If we reach this point, it's times to employ floyd and also enumerate path lengths.
    spaths = floyd(matrix)
    # Uncomment the below code to see the floyd algorithm printed in a nice format.
    # for i in range(len(spaths)):
    #     print(spaths[i])
    return find_most_bunnies(matrix, spaths, times_limit)
def floyd(matrix):
    """
    Floyd's algorithm, straight from a textbook. Floyd's algorithm transforms a weight matrix
    into a matrix of shortest paths, such that the shortest path from node M to node N is
    equal to matrix[m][n]
    :return: An array of shortest-path distance calculations.
    """
    n = len(matrix)
    spaths = deepcopy(matrix)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if spaths[i][k] + spaths[k][j] < spaths[i][j]:
                    spaths[i][j] = spaths[i][k] + spaths[k][j]
    return spaths
def find_most_bunnies(matrix, spaths, times_limit):
    """
    And now, yet more inefficient bruteforcing to solve our NP-Hard problem. Enumerate all possible subsets,
    and then evaluate all their permutations to find the most efficient path.
    :param matrix: The original weighted matrix we will analyze with our algorithm.
    :param spaths: An array of shortest paths generated by the floyd's algorithm.
    :param times_limit: The times limit each subset is tested against.
    :return: The lexicographically least subset of bunnies that can escape.
    """
    n = len(matrix)-2
    bunnyids = []
    for num in range(n):
        bunnyids.append(num)
    pset = powerset(bunnyids)
    pset = sorted(pset)
    # Now that I've got all our possible subsets, I can calculate the distance of each path and determine which is
    # optimal.
    optimal_bunnies = []
    for sub in pset:
        for permutation in permutations(sub):
            # print(permutation)
            subsum = 0
            prev = 0
            next = len(matrix)-1
            for bunnyid in permutation:
                next = bunnyid+1
                subsum += spaths[prev][next]
                prev = next
            subsum += spaths[prev][len(matrix)-1]
            if subsum <= times_limit and len(sub) > len(optimal_bunnies):
                optimal_bunnies = sub
                if len(optimal_bunnies) == n:
                    break
            else:
                # Either rescue takes too long, or lexicographically lesser solution of same length exists.
                pass
    return optimal_bunnies
def solution(times, times_limit):
    # I was told when I got my degree I would never be asked to solve the Traveling Salesman Problem.
    # I was misinformed, but this was fun!
    if len(times) <= 2:
        return []
    graph = matrix2graph(times)
    return bellman_ford(times, graph, times_limit, "Start")
if __name__ == '__main__':
    case1 = [[0, 1, 1, 1, 1],
             [1, 0, 1, 1, 1],
             [1, 1, 0, 1, 1],
             [1, 1, 1, 0, 1],
             [1, 1, 1, 1, 0]]
    print("\n\nCase 1: Provided test case 1.\nTimes limit: 3")
    for row in case1:
        print('', row)
    print("\n  Expected: [0, 1]\nCalculated:", str(solution(case1, 3)))
    print("\n\nCase 2: Provided test case 2.\nTimes limit: 1")
    case2 = [[0, 2, 2, 2, -1],
             [9, 0, 2, 2, -1],
             [9, 3, 0, 2, -1],
             [9, 3, 2, 0, -1],
             [9, 3, 2, 2, 0]]
    for row in case2:
        print('', row)
    print("\n  Expected: [1, 2]\nCalculated:", str(solution(case2, 1)))
    print("\n\nCase 3: Infinite negative cycle.\nTimes limit: -500")
    case3 = [[0, 2, 2, 2, -1],
             [9, 0, 2, 2, 0],
             [9, 3, 0, 2, 0],
             [9, 3, 2, 0, 0],
             [-1, 3, 2, 2, 0]]
    for row in case3:
        print('', row)
    print("\n  Expected: [0, 1, 2]\nCalculated:", str(solution(case3, -500)))
    print("\n\nCase 4: Max bunnies. None rescuable.\nTimes limit: 1")
    case4 = [[1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1]]
    for row in case4:
        print('', row)
    print("\n  Expected: []\nCalculated:", str(solution(case4, 1)))
    print("\n\nCase 5: One bunny.\nTimes limit: 2")
    case5 = [[1, 1, 1],
             [1, 1, 1],
             [1, 1, 1]]
    for row in case5:
        print('', row)
    print("\n  Expected: [0]\nCalculated:", str(solution(case5, 2)))
    print("\n\nCase 6: Multiple revisits.\nTimes limit: 10")
    case6 = [[0, 5, 11, 11, 1],
             [10, 0, 1, 5, 1],
             [10, 1, 0, 4, 0],
             [10, 1, 5, 0, 1],
             [10, 10, 10, 10, 0]]
    for row in case6:
        print('', row)
    print("\n  Expected: [0, 1]\nCalculated:", str(solution(case6, 10)))
    print("\n\nCase 7: Multiple Revisits 2.\nTimes limit: 5")
    case7 = [[0, 10, 10, 10, 1],
             [0, 0, 10, 10, 10],
             [0, 10, 0, 10, 10],
             [0, 10, 10, 0, 10],
             [1, 1, 1, 1, 0]]
    for row in case7:
        print('', row)
    print("\n  Expected: [0, 1]\nCalculated:", str(solution(case7, 5)))
    print("\n\nCase 8: Times travel.\nTimes limit: 1")
    case8 = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0]]
    for row in case8:
        print('', row)
    print("\n  Expected: [0, 1, 2]\nCalculated:", str(solution(case8, 1)))
    print("\n\nCase 9: No bunnies.\nTimes limit: 1")
    case9 = [[2, 2],
             [2, 2]]
    for row in case9:
        print('', row)
    print("\n  Expected: []\nCalculated:", str(solution(case9, 1)))
    print("\n\nCase 10: Backwards bunny path.\nTimes limit: 6")
    case10 = [[0, 10, 10, 1, 10],
              [10, 0, 10, 10, 1],
              [10, 1, 0, 10, 10],
              [10, 10, 1, 0, 10],
              [1, 10, 10, 10, 0]]
    for row in case10:
        print('', row)
    print("\n  Expected: [0, 1, 2]\nCalculated:", str(solution(case10, 6)))


# 2nd 

In [ ]:
# """
# Bringing a Gun to a Guard Fight
# ===============================
# # Uh-oh - you've been cornered by one of Commander Lambdas elite guards! Fortunately, you grabbed a
# beam weapon from an abandoned guardpost while you were running through the station, so you have a chance
# to fight your way out. But the beam weapon is potentially dangerous to you as well as to the elite guard:
#     its beams reflect off walls, meaning youll have to be very careful where you shoot to avoid bouncing
#     a shot toward yourself!
# # Luckily, the beams can only travel a certain maximum distance before becoming too weak to cause damage.
# You also know that if a beam hits a corner, it will bounce back in exactly the same direction. And of 
# course, if the beam hits either you or the guard, it will stop immediately (albeit painfully).
# # Write a function answer(dimensions, your_position, guard_position, distance) that gives an array of 2 
# integers of the width and height of the room, an array of 2 integers of your x and y coordinates in the 
# room, an array of 2 integers of the guard's x and y coordinates in the room, and returns an integer of 
# the number of distinct directions that you can fire to hit the elite guard, given the maximum distance
# that the beam can travel.
# # The room has integer dimensions [1 < x_dim <= 1000, 1 < y_dim <= 1000]. You and the elite guard are
# both positioned on the integer lattice at different distinct positions (x, y) inside the room such 
# that [0 < x < x_dim, 0 < y < y_dim]. Finally, the maximum distance that the beam can travel before 
# becoming harmless will be given as an integer 1 < distance <= 10000.
# # For example, if you and the elite guard were positioned in a room with dimensions [3, 2], 
# you_position [1, 1], guard_position [2, 1], and a maximum shot distance of 4, you could shoot in 
# seven different directions to hit the elite guard (given as vector bearings from your location): 
#     [1, 0], [1, 2], [1, -2], [3, 2], [3, -2], [-3, 2], and [-3, -2]. As specific examples, the shot
#     at bearing [1, 0] is the straight line horizontal shot of distance 1, the shot at bearing [-3, -2] 
#     bounces off the left wall and then the bottom wall before hitting the elite guard with a total shot
#     distance of sqrt(13), and the shot at bearing [1, 2] bounces off just the top wall before hitting
#     the elite guard with a total shot distance of sqrt(5).
# Languages
# =========
# To provide a Python solution, edit solution.py
# To provide a Java solution, edit solution.java
# Test cases
# ==========
# Inputs:
#     (int list) dimensions = [3, 2]
#     (int list) captain_position = [1, 1]
#     (int list) badguy_position = [2, 1]
#     (int) distance = 4
# Output:
#     (int) 7
# Inputs:
#     (int list) dimensions = [300, 275]
#     (int list) captain_position = [150, 150]
#     (int list) badguy_position = [185, 100]
#     (int) distance = 500
# Output:
#     (int) 9
# Time to solve: 360 hours.
# """

## working

In [ ]:
from __future__ import print_function
from collections import defaultdict
from itertools import product
import math

#checking the smallest number which devides a and b both
def gcd(a, b):
    a, b = abs(a), abs(b)
    while a != 0 and b != 0:
        a %= b
        if a == 0:
            break
        b %= a
        if b == 0:
            break
    return a + b

#number obtained by dividing by gcd both a and b
def fix(a, b):
    if a == 0 and b == 0:
        return 0, 0
    d = gcd(a, b)
    a //= d
    b //= d
    return a, b

#find distance between two points a and b
def norm(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)


def get_indexes(sp, tp, dimensions, distance, idx):
    indexes = set()
    for j in range(2):
        cur = tp[idx]
        i = j
        while abs(sp[idx] - cur) <= distance:
            indexes.add(cur)
            if i == 0:
                cur = -cur
            else:
                cur = 2 * dimensions[idx] - cur
            i = 1 - i
    return indexes


def get_points(sp, tp, dimensions, distance):
    xs = get_indexes(sp, tp, dimensions, distance, 0)
    ys = get_indexes(sp, tp, dimensions, distance, 1)
    return list(filter(lambda x: norm(x, sp) <= distance, product(xs, ys)))


def get_dirs(sp, ps):
    dirs = defaultdict(lambda: 0x3f3f3f3f)
    for p in ps:
        v = fix(p[0] - sp[0], p[1] - sp[1])
        dirs[v] = min(dirs[v], norm(sp, p))
    return dirs

def solution(dimensions, your_position, guard_position, distance):
    gps = get_points(your_position, guard_position, dimensions, distance)
    yps = get_points(your_position, your_position, dimensions, distance)
    # print(gps, yps)
    gdirs = get_dirs(your_position, gps)
    ydirs = get_dirs(your_position, yps)
    # print(gdirs, ydirs)
    ans = 0
    for k in gdirs:
        if k not in ydirs:
            ans += 1
        else:
            if gdirs[k] < ydirs[k]:
                ans += 1
    return ans

# 5th Challenge

In [1]:
# Disorderly Escape
# =================
# Oh no! You've managed to free the bunny prisoners and escape Commander Lambdas exploding space station, but 
# her team of elite starfighters has flanked your ship. If you dont jump to hyperspace, and fast, youl be shot
# out of the sky!
# # Problem is, to avoid detection by galactic law enforcement, Commander Lambda planted her space station in
# the middle of a quasar quantum flux field. In order to make the jump to hyperspace, you need to know the
# configuration of celestial bodies in the quadrant you plan to jump through. In order to do *that*, you need
# to figure out how many configurations each quadrant could possibly have, so that you can pick the optimal
# quadrant through which youll make your jump.
# # There's something important to note about quasar quantum flux fields' configurations: when drawn on a
# star grid, configurations are considered equivalent by grouping rather than by order. That is, for a given
# set of configurations, if you exchange the position of any two columns or any two rows some number of times,
# youll find that all of those configurations are equivalent in that way - in grouping, rather than order.
# # Write a function solution(w, h, s) that takes 3 integers and returns the number of unique, non-equivalent
# configurations that can be found on a star grid w blocks wide and h blocks tall where each celestial body
# has s possible states. Equivalency is defined as above: any two star grids with each celestial body in the
#     same state where the actual order of the rows and columns do not matter (and can thus be freely swapped
# around). Star grid standardization means that the width and height of the grid will always be between 1 
#     and 12, inclusive. And while there are a variety of celestial bodies in each grid, the number of states
#     of those bodies is between 2 and 20, inclusive. The solution can be over 20 digits long, so return it as
# a decimal string.  The intermediate values can also be large, so you will likely need to use at least 64-bit
# integers.
# # For example, consider w=2, h=2, s=2. We have a 2x2 grid where each celestial body is either in state 0 
# (for instance, silent) or state 1 (for instance, noisy).  We can examine which grids are equivalent by 
# swapping rows and columns.
# # 00
# # 00
# # In the above configuration, all celestial bodies are "silent" - that is, they have a state of 0 - so 
# any swap of row or column would keep it in the same state.
# # 00 00 01 10
# # 01 10 00 00
# # 1 celestial body is emitting noise - that is, has a state of 1 - so swapping rows and columns can put
# it in any of the 4 positions.  All four of the above configurations are equivalent.
# # 00 11
# # 11 00
# # 2 celestial bodies are emitting noise side-by-side.  Swapping columns leaves them unchanged, and 
# swapping rows simply moves them between the top and bottom.  In both, the *groupings* are the same: one 
#     row with two bodies in state 0, one row with two bodies in state 1, and two columns with one of each
#     state.
# # 01 10
# # 01 10
# # 2 noisy celestial bodies adjacent vertically. This is symmetric to the side-by-side case, but it is 
# different because there's no way to transpose the grid.
# # 01 10
# # 10 01
# # 2 noisy celestial bodies diagonally.  Both have 2 rows and 2 columns that have one of each state, so 
# they are equivalent to each other.
# 01 10 11 11
# 11 11 01 10
# 3 noisy celestial bodies, similar to the case where only one of four is noisy.
# 11
# 11
# 4 noisy celestial bodies.
# There are 7 distinct, non-equivalent grids in total, so solution(2, 2, 2) would return 7.
# Languages
# =========
# To provide a Java solution, edit Solution.java
# To provide a Python solution, edit solution.py
# Test cases
# ==========
# Your code should pass the following test cases.
# Note that it may also be run against hidden test cases not shown here.
# -- Java cases --
# Input:
# Solution.solution(2, 3, 4)
# Output:
#     430
# Input:
# Solution.solution(2, 2, 2)
# Output:
#     7
# -- Python cases --
# Input:
# solution.solution(2, 3, 4)
# Output:
#     430
# Input:
# solution.solution(2, 2, 2)
# Output:
#     7


## group theory, burnside lemma, polya enumeration theorem, cycle indices

In [44]:
from math import factorial
from collections import Counter
from fractions import gcd

def cycle_count(c, n):
    cc=factorial(n)
    
    for a, b in Counter(c).items():
        cc//=(a**b)*factorial(b)
    
    
    return cc


#generating a list of cycle partitions
def cycle_partitions(n, i=1):
     
    yield [n]
    for i in range(i, n//2 + 1):
        for p in cycle_partitions(n-i, i):
                    
            yield [i] + p
            
            
def solution(w, h, s):
    grid=0
    print("cycle partition:",list(cycle_partitions(w)))
    for cpw in cycle_partitions(w):
       
        for cph in cycle_partitions(h):
         
            m=cycle_count(cpw, w)*cycle_count(cph, h)
           
            grid+=m*(s**sum([sum([gcd(i, j) for i in cpw]) for j in cph]))
       
    return grid//(factorial(w)*factorial(h))




In [47]:
solution(2, 2, 2)

('cycle partition:', [[2], [1, 1]])


7

In [48]:

from copy import deepcopy
from itertools import permutations
# I am not happy with this implementation, due to a realization I have recently had:
#   My Bellman-Ford implementation is completely unnecessary. The Floyd algorithm can detect negative cycles, 
# and
# #   negative cycles are all I used Bellman-Ford for. Thus, with some refactoring, I could easily make this 
# more
#   efficient.
def powerset(list):
    """
    :param list: The list to generate subsets of.
    :return: A generator that produces all subsets of this set.
    """
    x = len(list)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, list) if i & mask]
        
        
        
def getneighbourindex(neighbour, graphsize):
    if neighbour == "Bulkhead":
        return graphsize - 1
    elif neighbour == "Start":
        return 0
    else:
        return int(neighbour)+1
    
    
def matrix2graph(matrix):
    """
    This helper function turns our matrix into a graph that's a little easier to work with using Bellman-Ford.
    :param matrix: The original matrix.
    :return: matrix in dictionary format.
    """
    keys = ["Start"]
    for num in range(1, len(matrix)-1):
        keys.append(num-1)
    keys.append("Bulkhead")
    graph = dict(zip(keys, matrix))
    return graph



# Step 1: Initialize graph
def initialize(graph, source):
    """
    Step 1 of the Bellman-Ford algorithm.
    """
    distance = {}
    predecessor = {}
    for node in graph:
        # We start off assuming all nodes are too far away!
        distance[node] = 1000
        predecessor[node] = None
    distance[source] = 0 # For the source we know how to reach
    return distance, predecessor



def relax(node, neighbour, graph, distance, predecessor):
    nidx = getneighbourindex(neighbour, len(graph))
    if distance[node] + graph[node][nidx] < distance[neighbour]:
        distance[neighbour] = distance[node] + graph[node][nidx]
        predecessor[neighbour] = node
        
        
        
def bellman_ford(matrix, graph, times_limit, source):
    dist, pred = initialize(graph, source)
    for num in range(len(graph)-1):
        for node in graph:
            temp = dict(graph)
            del temp[node]
            for neighbour in temp:
                # Step 2: Relax edges repeatedly
                relax(node, neighbour, graph, dist, pred)
    # Step 3: Check for negative-weight cycles
    for node in graph:
        for neighbour in graph:
            nidx = getneighbourindex(neighbour, len(graph))
            if dist[node] + graph[node][nidx] < dist[neighbour]:
                # We found a negative cycle. Since the door is open forever, free all the bunnies!~
                return [num for num in range(0, len(graph)-2)]
    # If we reach this point, it's times to employ floyd and also enumerate path lengths.
    spaths = floyd(matrix)
    # Uncomment the below code to see the floyd algorithm printed in a nice format.
    # for i in range(len(spaths)):
    #     print(spaths[i])
    return find_most_bunnies(matrix, spaths, times_limit)


def floyd(matrix):
    """
    Floyd's algorithm, straight from a textbook. Floyd's algorithm transforms a weight matrix
    into a matrix of shortest paths, such that the shortest path from node M to node N is
    equal to matrix[m][n]
    :return: An array of shortest-path distance calculations.
    """
    n = len(matrix)
    spaths = deepcopy(matrix)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if spaths[i][k] + spaths[k][j] < spaths[i][j]:
                    spaths[i][j] = spaths[i][k] + spaths[k][j]
    return spaths


def find_most_bunnies(matrix, spaths, times_limit):
    """
    And now, yet more inefficient bruteforcing to solve our NP-Hard problem. Enumerate all possible subsets,
    and then evaluate all their permutations to find the most efficient path.
    :param matrix: The original weighted matrix we will analyze with our algorithm.
    :param spaths: An array of shortest paths generated by the floyd's algorithm.
    :param times_limit: The times limit each subset is tested against.
    :return: The lexicographically least subset of bunnies that can escape.
    """
    n = len(matrix)-2
    bunnyids = []
    for num in range(n):
        bunnyids.append(num)
    pset = powerset(bunnyids)
    pset = sorted(pset)
    # Now that I've got all our possible subsets, I can calculate the distance of each path and determine which is
    # optimal.
    optimal_bunnies = []
    for sub in pset:
        for permutation in permutations(sub):
            # print(permutation)
            subsum = 0
            prev = 0
            next = len(matrix)-1
            for bunnyid in permutation:
                next = bunnyid+1
                subsum += spaths[prev][next]
                prev = next
            subsum += spaths[prev][len(matrix)-1]
            if subsum <= times_limit and len(sub) > len(optimal_bunnies):
                optimal_bunnies = sub
                if len(optimal_bunnies) == n:
                    break
            else:
                # Either rescue takes too long, or lexicographically lesser solution of same length exists.
                pass
    return optimal_bunnies

def solution(times, times_limit):
    # I was told when I got my degree I would never be asked to solve the Traveling Salesman Problem.
    # I was misinformed, but this was fun!
    if len(times) <= 2:
        return []
    graph = matrix2graph(times)
    return bellman_ford(times, graph, times_limit, "Start")

In [51]:
times = [[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]
times_limit = 3

In [52]:
solution(times, times_limit)

[0, 1]

In [1]:
dict1={"a": "ball","b":"val"}

In [3]:
dict1["a"]="cat"

In [4]:
dict1

{'a': 'cat', 'b': 'val'}

In [5]:
set1={"cat","bat"}

In [12]:
set1.get(0)

AttributeError: 'set' object has no attribute 'get'